#### LangSmith Starter

# LangSmith Starter

LangSmith is an advanced open-source library tailored for AI Engineers, focusing on enabling seamless development and deployment of language-based AI systems. With an emphasis on modularity and interoperability, LangSmith simplifies the integration of natural language processing (NLP) capabilities into AI workflows, offering intuitive tools for fine-tuning, optimization, and interaction. It empowers developers to switch effortlessly between language models, customize pipelines, and deploy scalable, production-ready applications with minimal friction.

Super important note, you need to run LangSmith on more then 0.1.45 and be using the current API keys and not the legacy keys, if you do not do this you will recieve a 401 error message saying the token you are using is invalid.

This below is the enviroment data for enabling tracing, very important that you do not mess around with this as even replacing a singular character can break and then you will not be able to trace anything.

In [1]:
import os
from getpass import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass("Enter LangSmith API Key: ")
os.environ["LANGCHAIN_PROJECT"] = "aai-langchain-course-langsmith-starter-openai"

For this tutorial we will be using OpenAI, however this is basically the exact same for llama just with a few changes.

In [2]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

openai_model = "gpt-4o-mini"

Langsmith by default will trace all the usual langchain based LLM calls, which in the tutorials case is super useful as we won't have to edit much code to get traces working.

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.0, model=openai_model)

In [4]:
llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-04a75541-53a4-4d38-b7f9-1be2b4a15031-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

This is settup for the non-langchain function.

In [7]:
import openai

client = openai

Now we want to use a non langchain related function, and these are not automatically traced by langsmith, so instead, we have to add the traceable decorator.

In [12]:
from langsmith import traceable

@traceable
def generate_response(question: str):

    complete_messages = [
    {"role": "system", "content": "You are a happy assistant"},
    {"role": "user", "content": question}
    ]

    return client.chat.completions.create(
        model = openai_model,
        messages = complete_messages,
        temperature = 0.1
    )

In [13]:
generate_response("How are you today?")

ChatCompletion(id='chatcmpl-AkCKHY4A66FAYToGl9lcN7YOVGALe', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm feeling great, thank you! How about you? How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735573833, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0aa8d3e20b', usage=CompletionUsage(completion_tokens=19, prompt_tokens=21, total_tokens=40, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

We can also add data into the traceable by editing its paramaters, this includes metadata, and what we will change, the function name.

In [14]:
from langsmith import traceable

@traceable(name="OpenAI Response")
def generate_second_response(question: str):

    complete_messages = [
    {"role": "system", "content": "You are a happy assistant"},
    {"role": "user", "content": question}
    ]

    return client.chat.completions.create(
        model = openai_model,
        messages = complete_messages,
        temperature = 0.1
    )

In [15]:
generate_second_response("How are you today?")

ChatCompletion(id='chatcmpl-AkCUBhB9ZPITBWIjGrPcbtXMWgfuZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm feeling great, thank you! How about you? How can I assist you today?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735574447, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0aa8d3e20b', usage=CompletionUsage(completion_tokens=19, prompt_tokens=21, total_tokens=40, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

Now you can view all of this over at the LangSmith website, under the tracing projects.